In [1]:
import socket
import threading
import os

In [ ]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(('www.google.com', 80))
s.send(b'GET / HTTP/1.1\r\nHost: www.google.com\r\nConnection: close\r\n\r\n')
buffer = []
while True:
    d = s.recv(1024)
    if d:
        buffer.append(d)
    else:
        break
data = b''.join(buffer)
s.close()
header, html = data.split(b'\r\n\r\n', 1)
print(header.decode('utf-8'))
with open('sina.html', 'wb') as f:
    f.write(html)

In [ ]:
def tcplink(sock, addr):
    print('Accept new connection from %s:%s...' % addr)
    sock.send(b'Welcome!')
    while True:
        data = sock.recv(1024)
        time.sleep(1)
        if not data or data.decode('utf-8') == 'exit':
            break
        sock.send(('Hello, %s!' % data.decode('utf-8')).encode('utf-8'))
    sock.close()
    print('Connection from %s:%s closed.' % addr)

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind(('127.0.0.1', 9999))
s.listen(5)
print('Waiting for connection...')
while True:
    # receive a new connect:
    sock, addr = s.accept()
    # create new thread to cope with tcp connection:
    t = threading.Thread(target=tcplink, args=(sock, addr))
    t.start()


In [ ]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# create connection:
s.connect(('127.0.0.1', 9999))
# receive hello information:
print(s.recv(1024).decode('utf-8'))
for data in [b'Michael', b'Tracy', b'Sarah']:
    # send data:
    s.send(data)
    print(s.recv(1024).decode('utf-8'))
s.send(b'exit')
s.close()

In [6]:
os.getpid()
os.fork()

5563

In [2]:
import findspark
findspark.init()

In [9]:
from pyspark import SparkConf, SparkContext
sc = SparkContext()

intRDD = sc.parallelize([3,1,2,5,5])
stringRDD = sc.parallelize(['Apple','Orange','Grape','Banana','Apple'])

print (intRDD.collect())
print (stringRDD.collect())

print(intRDD.map(lambda x:x+1).collect())

sRDD = intRDD.randomSplit([0.3,0.7])
print(len(sRDD))
print(sRDD[0].collect())
print(sRDD[1].collect())

kvRDD1 = sc.parallelize([(3,4),(3,6),(5,6),(1,2)])

print(kvRDD1.keys().collect())
print (kvRDD1.values().collect())
print (kvRDD1.filter(lambda x:x[1] < 5).collect())
print (kvRDD1.mapValues(lambda x:x**2).collect())
print (kvRDD1.sortByKey(False).collect())
print(kvRDD1.reduceByKey(lambda x,y:x*y).collect())

kvRDD1 = sc.parallelize([(3,4),(3,6),(5,6),(1,2)])
kvRDD2 = sc.parallelize([(3,8),(2,7)])
print (kvRDD1.join(kvRDD2).collect())
print (kvRDD1.rightOuterJoin(kvRDD2).collect())
print (kvRDD1.countByKey())
print(kvRDD1.lookup(3))

kvRDD1.persist()

from pyspark.storagelevel import StorageLevel
kvRDD1.unpersist()

In [7]:
import requests
from multiprocessing.dummy import Pool as ThreadPool
import time

In [8]:
start_time = time.time()
urls = [
  'http://www.python.org', 
  'http://www.python.org/about/',
  'http://www.onlamp.com/pub/a/python/2003/04/17/metaclasses.html',
  'http://www.python.org/doc/',
  'http://www.python.org/download/',
  'http://www.python.org/getit/',
  'http://www.python.org/community/',
  'https://wiki.python.org/moin/',
]
pool = ThreadPool(4)
results = pool.map(requests.get, urls)
pool.close()
pool.join()
print("--- %s seconds ---" % (time.time() - start_time))


--- 1.3073816299438477 seconds ---


In [9]:
start_time = time.time()
results = []
for url in urls:
    results.append(requests.get(url))
print("--- %s seconds ---" % (time.time() - start_time))


--- 3.3689873218536377 seconds ---


In [11]:
zip([1,2,3],[4,5,6])

In [65]:
import xml.etree.ElementTree as ET
import csv

tree = ET.ElementTree(file='ginnywise.xml')

root = tree.getroot()
root.tag, root.attrib

outputFile = 'ginnywise.csv'
with open(outputFile, "w") as fp:
    wr = csv.writer(fp, dialect='excel', quotechar = '"')
    wr.writerow(['locationName','stationId','obsTime','測站氣壓','溫度','相對濕度','風速','風向','降水量'])
for loc in range(len(root)): # 每個location
    timeNum = len(root[loc][2]) # 每個location的time數量
    for time in range(1,timeNum):
        print(loc, time)
        tmpList = [root[loc][0].text,
                 root[loc][1].text,
                 root[loc][2][time].find('obsTime').text,
                 root[loc][2][time][1][1][0].text,
                 root[loc][2][time][2][1][0].text,
                 root[loc][2][time][3][1][0].text,
                 root[loc][2][time][4][1][0].text,
                 root[loc][2][time][5][1][0].text,
                 root[loc][2][time][6][1][0].text]
        with open(outputFile, "a") as fp:
            wr = csv.writer(fp, dialect='excel', quotechar = '"')
            wr.writerow(tmpList)